In [2]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.nn.functional as F

import dgl
import os
import sys
import pickle
import scipy.sparse as ss
print(os.getcwd())

sys.path.append('.')
sys.path.append('../')
sys.path.append('../../')
from src.utils.functions import load_model, load_sequence, prepare_batches, add_features_dataset, normalise_statistics, dist_labels_to_changepoint_labels, dist_labels_to_changepoint_labels_adjusted
from src.utils.misc import get_device
from src.detect import compute_sgnn_similarity
from src.utils.baselines import laplacian_spectrum_similarity, NCPD, CUMSUM, CUMSUM_2, avg_deltacon_similarity, avg_wl_distance, avg_frobenius_distance, avg_procrustes_distance, \
    distance_procrustes_LE, evaluate_baseline
from src.utils.graphs import laplacian_embeddings, norm_laplacian, laplacian, degree_matrix
import torch
import matplotlib.pyplot as plt
import numpy as np
import scipy
import grakel
from grakel.kernels import WeisfeilerLehman, VertexHistogram
from src.utils.metrics import find_best_threshold, adjust_predicts_donut, binary_metrics_adj, best_f1_search_grid, compute_ari
from pathlib import PosixPath
import json



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/maruanottoni/home/sgnn/notebooks


ModuleNotFoundError: No module named 'ruptures'